In [3]:
import spacy
from spacy import displacy

In [4]:
nlp = spacy.load("en_core_web_sm")

In [20]:
text = "Apple, This is first sentence. and  Google this is another one. Here third one is 3rd"

In [21]:
doc = nlp(text)

In [22]:
doc

Apple, This is first sentence. and  Google this is another one. Here third one is 3rd

In [23]:
for token in doc:
    print(token)

Apple
,
This
is
first
sentence
.
and
 
Google
this
is
another
one
.
Here
third
one
is
3rd


In [24]:
sent = nlp.create_pipe('sentencizer')

In [30]:
nlp.add_pipe(sent, before='parser')

In [32]:
doc = nlp(text)

In [33]:
for sent in doc.sents:
    print(sent)

Apple, This is first sentence.
and  Google this is another one.
Here third one is 3rd


In [34]:
from spacy.lang.en.stop_words import STOP_WORDS

In [35]:
stopwords = list(STOP_WORDS)

In [36]:
#stopwords

In [37]:
len(stopwords)

326

In [38]:
for token in doc:
    if token.is_stop == False:
        print(token)

Apple
,
sentence
.
 
Google
.
3rd


### Lemmatization

In [39]:
doc = nlp('run runs running runner')

In [41]:
for lem in doc:
    print(lem.text, lem.lemma_)

run run
runs run
running run
runner runner


### POS

In [43]:
doc = nlp('All is well at your end!')

In [46]:
for token in doc:
    print(token.text, token.pos_)

All DET
is AUX
well ADV
at ADP
your DET
end NOUN
! PUNCT


In [52]:
displacy.render(doc, style='dep', options = {'compact':True, 'distance':100})

### Entity Detection

In [59]:
doc = nlp('New York City on Tuesday declared a public health emergency and ordered mandatory measles vaccinations amid an outbreak, becoming the latest national flash point over refusals to inoculate against dangerous diseases. At least 285 people have contracted measles in the city since September, mostly in Brooklyn’s Williamsburg neighborhood. The order covers four Zip codes there, Mayor Bill de Blasio (D) said Tuesday. The mandate orders all unvaccinated people in the area, including a concentration of Orthodox Jews, to receive inoculations, including for children as young as 6 months old. Anyone who resists could be fined up to $1,000.')

In [60]:
doc

New York City on Tuesday declared a public health emergency and ordered mandatory measles vaccinations amid an outbreak, becoming the latest national flash point over refusals to inoculate against dangerous diseases. At least 285 people have contracted measles in the city since September, mostly in Brooklyn’s Williamsburg neighborhood. The order covers four Zip codes there, Mayor Bill de Blasio (D) said Tuesday. The mandate orders all unvaccinated people in the area, including a concentration of Orthodox Jews, to receive inoculations, including for children as young as 6 months old. Anyone who resists could be fined up to $1,000.

In [61]:
displacy.render(doc, style='ent')

### Text Classification

In [62]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report, confusion_matrix

In [68]:
data_yelp = pd.read_csv('F:/MLP/NLP/datasets/yelp_labelled.txt',sep='\t', header=None)

In [69]:
data_yelp.head()

,0,1
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [70]:
columns_name = ['Review','Sentiment']
data_yelp.columns = columns_name

In [71]:
data_yelp.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [72]:
data_yelp.shape

(1000, 2)

In [73]:
data_amazon = pd.read_csv('F:/MLP/NLP/datasets/amazon_cells_labelled.txt',sep='\t', header=None)

In [74]:
data_amazon.head()

,0,1
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [75]:
columns_name = ['Review','Sentiment']
data_amazon.columns = columns_name

In [76]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [77]:
data_amazon.shape

(1000, 2)

In [78]:
data_imdb = pd.read_csv('F:/MLP/NLP/datasets/imdb_labelled.txt',sep='\t', header=None)

In [79]:
columns_name = ['Review','Sentiment']
data_imdb.columns = columns_name

In [80]:
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [81]:
data_imdb.shape

(748, 2)

In [82]:
data = data_yelp.append([data_amazon, data_imdb], ignore_index=True)

In [84]:
data.shape

(2748, 2)

In [85]:
data.head()

,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [87]:
data['Sentiment'].value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

In [89]:
data.isnull().sum()

Review       0
Sentiment    0
dtype: int64

### Tokenization

In [91]:
import string

In [92]:
punct = string.punctuation

In [93]:
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [96]:
def data_cleaning(sentence):
    doc = nlp(sentence)
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [97]:
data_cleaning('Hello how are you.')

['hello']

### Vectorization Feature Engineering(TF-IDF)

In [98]:
from sklearn.svm import LinearSVC

In [107]:
tfidf = TfidfVectorizer(tokenizer = data_cleaning)
classifier = LinearSVC()

In [108]:
X = data['Review']
y = data['Sentiment']

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X ,y, test_size=0.2, random_state=42)

In [110]:
X_train.shape, X_test.shape

((2198,), (550,))

In [111]:
X_train.head()

1670    its extremely slow and takes forever to do any...
2487    There are many continuity errors: one other us...
1124                                   Gets the job done.
2480                         This is an excellent film.  
480     We asked for the bill to leave without eating ...
Name: Review, dtype: object

In [112]:
clf = Pipeline([('tfidf',tfidf),('clf',classifier)])

In [113]:
clf.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function data_cleaning at 0x0000020F6E84CCA8>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=

In [114]:
y_pred = clf.predict(X_test)

In [115]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.77      0.82      0.80       285
           1       0.79      0.74      0.77       265

    accuracy                           0.78       550
   macro avg       0.78      0.78      0.78       550
weighted avg       0.78      0.78      0.78       550



In [116]:
confusion_matrix(y_test, y_pred)

array([[233,  52],
       [ 68, 197]], dtype=int64)

In [117]:
clf.predict(['Wow, this is amazing lesson'])

array([1], dtype=int64)

In [118]:
clf.predict(['Wow, this     sucks'])

array([0], dtype=int64)

In [119]:
clf.predict(['Worth of watching it'])

array([1], dtype=int64)